# Train

In [1]:
import pickle
import numpy as np
from sklearn.utils import shuffle

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import Adam

Using TensorFlow backend.


## 1 Reload data

In [2]:
def reload_data(pickle_file):
    print('reload: ', pickle_file)
    with open(pickle_file, 'rb') as f:
        pickle_data = pickle.load(f)
        X_train = pickle_data['X_train']
        y_train = pickle_data['y_train']
        del pickle_data  # Free up memory
    return X_train, y_train

X_train, y_train = reload_data('./pre-data.pickle')
X_train, y_train = shuffle(X_train, y_train)
print('X_train shape: ', X_train.shape, 'y_train shape: ',y_train.shape)

reload:  ./pre-data.pickle
X_train shape:  (24108, 80, 80, 3) y_train shape:  (24108,)


## 2 Model architecture

In [3]:
nvidia = Sequential()
nvidia.add(Lambda(lambda x: x/255. - 0.5, input_shape=(80, 80, 3)))
nvidia.add(Cropping2D(cropping=((35, 13), (0, 0))))
nvidia.add(Convolution2D(24, 3, 3, subsample=(2, 2), activation='relu'))
nvidia.add(Convolution2D(36, 3, 3, subsample=(2, 2), activation='relu'))
nvidia.add(Convolution2D(48, 3, 3, activation='relu'))
nvidia.add(Convolution2D(64, 3, 3, activation='relu'))
nvidia.add(Convolution2D(64, 3, 3, activation='relu'))
nvidia.add(Dropout(0.5))
nvidia.add(Flatten())
nvidia.add(Dense(100))
nvidia.add(Dense(50))
nvidia.add(Dense(10))
nvidia.add(Dense(1))

## 3 Train the model

In [4]:
%%time
# Hyperparameters
LEARNING_RATE = 1e-4
EPOCHS = 5

# Training
nvidia.compile(loss='mse', optimizer=Adam(LEARNING_RATE))
nvidia.fit(X_train, y_train, validation_split=0.2, 
           shuffle=True, nb_epoch=EPOCHS)
nvidia.save('model.h5')

Train on 19286 samples, validate on 4822 samples
Epoch 1/5
19286/19286 [==============================] - 130s - loss: 0.0131 - val_loss: 0.0111
Epoch 2/5
19286/19286 [==============================] - 10s - loss: 0.0110 - val_loss: 0.0108
Epoch 3/5
19286/19286 [==============================] - 12s - loss: 0.0105 - val_loss: 0.0104
Epoch 4/5
19286/19286 [==============================] - 12s - loss: 0.0102 - val_loss: 0.0100
Epoch 5/5
19286/19286 [==============================] - 12s - loss: 0.0099 - val_loss: 0.0102


## 4 Train the model with generator

In [4]:
import csv
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [26]:
lines = []
with open('./sample_data/data/driving_log.csv') as sample_file:
    sample_file.readline()
    reader = csv.reader(sample_file)
    for line in reader:
        lines.append(line)
print(len(lines))

train_lines, valid_lines = train_test_split(lines, test_size=0.2, random_state=1)
print(len(train_lines), len(valid_lines))

8036
6428 1608


In [31]:
BATCH_SIZE = 128
def generator(samples, batch_size):
    ns = len(samples)
    while True:
        indices = np.random.randint(0, ns, batch_size)
#         batch_samples = samples[indices]
#         print(batch_samples)
        images = []
        angles = []
        for indix in indices:
            sample = samples[indix]
            center_path = './sample_data/data/IMG/' + sample[0].split('/')[-1]
            center_img = plt.imread(center_path)
            center_img = cv2.resize(center_img, (80,80))
            center_angle = float(line[3])
            images.append(center_img)
            angles.append(center_angle)

            left_path = './sample_data/data/IMG/' + sample[1].split('/')[-1]
            left_img = plt.imread(left_path)
            left_img = cv2.resize(left_img, (80,80))
            left_angle = float(line[3]) + 0.10
            images.append(left_img)
            angles.append(left_angle)

            right_path = './sample_data/data/IMG/' + sample[2].split('/')[-1]
            right_img = plt.imread(right_path)
            right_img = cv2.resize(right_img, (80,80))
            right_angle = float(line[3]) - 0.10
            images.append(right_img)
            angles.append(right_angle)

        yield np.array(images), np.array(angles)
            
train_generator = generator(train_lines, BATCH_SIZE)
valid_generator = generator(valid_lines, BATCH_SIZE)

In [32]:
%%time
# Hyperparameters
LEARNING_RATE = 5e-4
EPOCHS = 5
SAMPLES_PER_EPOCH = 6*BATCH_SIZE*int(len(train_lines)/BATCH_SIZE)
NB_VAL_SAMPLES = 3*BATCH_SIZE*int(len(valid_lines)/BATCH_SIZE)

# Training
nvidia.compile(loss='mse', optimizer=Adam(LEARNING_RATE))
nvidia.fit_generator(train_generator, 
                     samples_per_epoch=SAMPLES_PER_EPOCH,
                     validation_data=valid_generator, 
                     nb_val_samples=NB_VAL_SAMPLES, 
                     nb_epoch=EPOCHS, verbose=1)
nvidia.save('modelg.h5')

Epoch 1/5
38400/38400 [==============================] - 53s - loss: 0.0017 - val_loss: 0.0020
Epoch 2/5
38400/38400 [==============================] - 48s - loss: 0.0012 - val_loss: 0.0021
Epoch 3/5
38400/38400 [==============================] - 48s - loss: 0.0011 - val_loss: 0.0020
Epoch 4/5
38400/38400 [==============================] - 49s - loss: 0.0010 - val_loss: 0.0020
Epoch 5/5
38400/38400 [==============================] - 49s - loss: 9.5612e-04 - val_loss: 0.0020
CPU times: user 5min 1s, sys: 13.3 s, total: 5min 15s
Wall time: 4min 9s
